# Embeddings and Similarity with DistilBERT

DS 5001 Text as Data

In [2]:
# pip install transformers

In [32]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load DistilBERT

In [21]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval();

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# Define a helper to compute embeddings

In [22]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use [CLS] token embedding

# Load a few lines from a text

In [23]:
src_file = "data/frankenstein.txt"

In [47]:
DOC = pd.DataFrame([line.strip() for line in open(src_file, 'r').readlines() if line.strip() != ""])
DOC.columns = ['line_str']
DOC.index.name = 'line_num'

In [48]:
DOC

,line_str
line_num,
0,Letter 1
1,"_To Mrs. Saville, England._"
2,"St. Petersburgh, Dec. 11th, 17—."
3,You will rejoice to hear that no disaster has accompanied the
4,commencement of an enterprise which you have regarded with such evil
...,...
6382,"will sleep in peace, or if it thinks, it will not surely think thus."
6383,Farewell.”
6384,"He sprang from the cabin-window as he said this, upon the ice raft"


In [24]:
# with open(src_file, encoding="utf-8") as f:
#     lines = [line.strip() for line in f if line.strip() != ""]

In [49]:
# sample_passages = lines[:5]  # Choose first five paragraphs for demo

In [50]:
# sample_passages

# Compute embeddings

In [ ]:
EMB = DOC.line_str.apply(get_embedding)
EMB = pd.DataFrame([pd.Series(row[0]) for row in EMB.values])

# Pairwise similarity

In [34]:
sim_matrix = cosine_similarity(embeddings)

In [77]:
SIMS = pd.DataFrame(cosine_similarity(EMB))

In [78]:
SIMS

,0,1,2,3,4,5,6,7,8,9,...,6377,6378,6379,6380,6381,6382,6383,6384,6385,6386
0,1.000000,0.915007,0.841304,0.935454,0.856641,0.937932,0.932358,0.959834,0.914672,0.926192,...,0.938372,0.866512,0.864856,0.896121,0.866028,0.873589,0.956218,0.897252,0.928944,0.897847
1,0.915007,1.000000,0.858061,0.907682,0.864672,0.908874,0.917725,0.917606,0.905153,0.907627,...,0.904148,0.872664,0.885705,0.880915,0.873422,0.866197,0.916288,0.892079,0.890111,0.886070
2,0.841304,0.858061,1.000000,0.847964,0.856526,0.849992,0.843637,0.849841,0.855166,0.861975,...,0.830672,0.836010,0.844545,0.827492,0.825361,0.834220,0.827751,0.849063,0.835795,0.834834
3,0.935454,0.907682,0.847964,1.000000,0.906610,0.946110,0.965291,0.961159,0.938419,0.944174,...,0.966069,0.925255,0.931479,0.929550,0.914747,0.926476,0.957180,0.939144,0.949216,0.901663
4,0.856641,0.864672,0.856526,0.906610,1.000000,0.886968,0.906106,0.891526,0.895304,0.885050,...,0.867511,0.889048,0.901962,0.873760,0.887367,0.908602,0.853779,0.892870,0.868122,0.855968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382,0.873589,0.866197,0.834220,0.926476,0.908602,0.903611,0.899346,0.907966,0.926397,0.918466,...,0.912599,0.947066,0.929860,0.921910,0.940568,1.000000,0.884384,0.934326,0.908069,0.901094
6383,0.956218,0.916288,0.827751,0.957180,0.853779,0.938879,0.947704,0.969408,0.926612,0.940637,...,0.972868,0.887971,0.889336,0.912746,0.877551,0.884384,1.000000,0.909372,0.938497,0.901100
6384,0.897252,0.892079,0.849063,0.939144,0.892870,0.915351,0.914432,0.927746,0.940385,0.944473,...,0.943030,0.934193,0.923512,0.940680,0.916441,0.934326,0.909372,1.000000,0.961094,0.905422
6385,0.928944,0.890111,0.835795,0.949216,0.868122,0.941032,0.927222,0.951546,0.942536,0.952051,...,0.950941,0.912986,0.907493,0.936688,0.902625,0.908069,0.938497,0.961094,1.000000,0.915980


In [80]:
DOC.loc[6384]

line_str    He sprang from the cabin-window as he said this, upon the ice raft
Name: 6384, dtype: object

In [86]:
DOC.loc[EMB.loc[6384].sort_values(ascending=False).head().index]

,line_str
205,regions I am about to explore.
588,"hair was the brightest living gold, and despite the poverty of her"
360,so attractive and amiable.
678,"indiscriminately. I confess that neither the structure of languages,"
250,"recording it, although it is very probable that you will see me before"


In [36]:
similarity_df = pd.DataFrame(sim_matrix, index=[f"P{i+1}" for i in range(len(sample_passages))],
                              columns=[f"P{i+1}" for i in range(len(sample_passages))])

In [37]:
for i, p in enumerate(sample_passages):
    print(f"P{i+1}: {p[:80]}...")

P1: Letter 1...
P2: _To Mrs. Saville, England._...
P3: St. Petersburgh, Dec. 11th, 17—....
P4: You will rejoice to hear that no disaster has accompanied the...
P5: commencement of an enterprise which you have regarded with such evil...


In [ ]:
print("\nCosine Similarity Matrix:")
display(similarity_df.round(2))

Sample Passages:
P1: Letter 1...
P2: _To Mrs. Saville, England._...
P3: St. Petersburgh, Dec. 11th, 17—....
P4: You will rejoice to hear that no disaster has accompanied the...
P5: commencement of an enterprise which you have regarded with such evil...

Cosine Similarity Matrix:


,P1,P2,P3,P4,P5
P1,1.00,0.92,0.84,0.94,0.86
P2,0.92,1.00,0.86,0.91,0.86
P3,0.84,0.86,1.00,0.85,0.86
P4,0.94,0.91,0.85,1.00,0.91
P5,0.86,0.86,0.86,0.91,1.00


In [12]:
sample_passages

['The Project Gutenberg EBook of Persuasion, by Jane Austen',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net']